# Рекомендация тарифов клиенту компании сотовой связи.

На уже предобработанных данных требуется построить модель для задачи классификации, которая выберет подходящий тариф клиенту компании сотовой связи.

Требуется построить модель с максимально большим значением *accuracy*, минимально допустимое значение метрики - 0.75.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('/tariff_recommendation/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Открыл файл и изучил данные в нём. Так как предобработка уже сделана - никаких аномалий мной не выявлено. Определён целевой признак - столбец `is_ultra`. Исходя из этого будет происходить дальнейшая работа.

## Разделим данные на выборки

In [5]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [6]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target) 

In [7]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

In [8]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


Разделил данные на три выборки - обучающую, валидационную и тестовую, в соотношении 3:1:1. Выполнил проверку, что данные разделены правильно. 

## Исследуем модели

**Модель Дерево решений.**

In [9]:
for depth in range(1, 11):
    model = model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7402799377916018
max_depth = 2 : 0.7729393468118196
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7853810264385692
max_depth = 6 : 0.7744945567651633
max_depth = 7 : 0.7869362363919129
max_depth = 8 : 0.80248833592535
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7729393468118196


При глубине дерева равной восьми достигнута максимальная accuracy - 0.80248833592535.

**Модель Случайный лес.**

In [10]:
best_model = None
best_est = 0
best_result = 0
for est in range(1, 24):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_est = est
        best_result = result

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'при', best_est, 'деревьях в лесу')

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625 при 21 деревьях в лесу


**Модель Логистическая регрессия.**

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)

prediction_valid = model.predict(features_valid)
result = accuracy_score(prediction_valid, target_valid)

print('Accuracy:', result)

Accuracy: 0.7387247278382582


Провел исследование качества разных моделей - Дерево решений, Случайный лес и Логистическая регрессия.

В Дереве решений изменил гиперпараметр  max_depth от 1 до 10 в цикле. Результаты вывел на экран, accuracy во всех вариантах модели больше минимально допустимой - 0.75. наилучший вариант - 8, с показателем 0.80248833592535.

В Модели Случайный лес изменил гиперпараметр n_estimators в цикле от 1 до 25. Наилучшая модель показала accuracy - 0.7993779160186625 при 21 деревьях.

В модели Логистическая регрессия показатель accuracy оказался даже ниже допустимого - 0.7387247278382582.

Наилучший результат показала модель Дерево решений. 

Для проверки модели на тестовой выборке будем использовать модель Дерево решений.

## Проверим модель на тестовой выборке

In [12]:
features_uni = pd.concat([features_train,features_valid])

In [13]:
target_uni = pd.concat([target_train,target_valid])

In [14]:
print(features_uni.shape)
print(target_uni.shape)

(2571, 4)
(2571,)


In [15]:
model = DecisionTreeClassifier(random_state=12345, max_depth=8)
model.fit(features_uni, target_uni)
result = model.score(features_test, target_test)

print("Accuracy модели на тестовой выборке:", result)

Accuracy модели на тестовой выборке: 0.807153965785381


Accuracy модели Дерево решений на тестовой выборке: 0.807153965785381.

## Проверим модели на адекватность

In [16]:
model = DummyClassifier(strategy='prior', random_state=12345)
model.fit(features_uni, target_uni)
result = model.score(features_test, target_test)
print('Accuracy проверочной модели:', result)                     

Accuracy проверочной модели: 0.6936236391912908


Для проверки модели на адекватность использовал модель DummyClassifier. Accuracy проверочной модели меньше чем у выбранной нами модели Дерево решений. Значит наша модель прошла проверку на адекватность.

**Вывод**

Предоставленный файл, прошедший предобработку, никаких аномалий не имеет.

Данные разбили на три выборки - обучающую, валидационную и тестовую в пропорции 3:1:1.

Были исследованы три модели - DecisionTreeClassifier (Дерево решений), RandomForestClassifier (Случайный лес) и LogisticRegression (Логистическая регрессия).

Наилучший результат показала модель DecisionTreeClassifier (Дерево решений) - accuracy - 0.80248833592535.

Выполнил объединение обучающей и валидационной выборок для проверки модели на тестовой выборке.

Модель DecisionTreeClassifier была выбрана для проверки на тестовой выборке, которая показала accuracy - 0.807153965785381.

Провели проверку на адекватность с помощью модели DummyClassifier, accuracy которой оказалось - 0.6936236391912908.

Исходя из этого делаем вывод, что выбранная нами модель эффективна и адекватна.